In [73]:
import time as clock
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from IPython.display import clear_output, display
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

In [74]:
def thermal_conductivity_equation(x):
    return (2 + torch.exp(-4*(torch.pi**2)*x[0])*torch.sin(2*torch.pi*x[1]) + torch.exp(-16*(torch.pi**2)*x[0])*torch.cos(4*torch.pi*x[1]))

In [75]:
DEVICE = 'cpu'
N = 100

In [76]:
x_physics = torch.load(f'data/x_{N}.pt').requires_grad_(True).to(DEVICE)
time_physics = torch.load(f'data/t_{N}.pt').requires_grad_(True).to(DEVICE)
points_for_pinn_model = torch.stack((time_physics, x_physics), -1).to(DEVICE)
boundary_condition_points = torch.stack((torch.zeros(N).to(DEVICE), x_physics), -1).to(DEVICE)
periodic_points_x0 = torch.stack((time_physics, torch.zeros(N).to(DEVICE)), -1).to(DEVICE)
periodic_points_x1 = torch.stack((time_physics, torch.ones(N).to(DEVICE)), -1).to(DEVICE)

In [77]:
class PINN(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32, num_layers=4, output_dim=1):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.U = nn.Linear(input_dim, hidden_dim)
        self.V = nn.Linear(input_dim, hidden_dim)

        self.hidden_layers = nn.ModuleList(
            [nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers - 1)]
        )

        self.output_layer = nn.Linear(hidden_dim, output_dim)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=1.0)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        U = torch.tanh(self.U(x))
        V = torch.tanh(self.V(x))

        h = U * V

        for i, layer in enumerate(self.hidden_layers):
            h_new = torch.relu(layer(h))
            if i % 2 == 1 and i > 0:
                h = h + h_new
            else:
                h = h_new

        T = self.output_layer(h)
        return T

# SVM

In [78]:
def check_current_situation_others(svm_model, scaler, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = svm_model.predict(scaler.transform(test_points))

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()
    
def check_current_situation_xgb(xgb, t):
    x = np.linspace(0, 1, 100)
    time = t * np.ones(100)
    real_temp = thermal_conductivity_equation([torch.tensor(time), torch.tensor(x)])

    test_points = np.stack((time, x), axis = -1)

    svm_pred = xgb.predict(test_points)

    plt.plot(x, svm_pred, label=f'SVM prediction with t = {t}')
    plt.plot(x, real_temp, label=f'Analytical solution with t = {t}')
    plt.grid()
    plt.xlabel('x')
    plt.ylabel(f'T(t={t}, x)')

    plt.plot()
    plt.legend()
    plt.show()

In [79]:
x_svm = x_physics.cpu().detach().numpy()
time_svm = time_physics.cpu().detach().numpy()

X_train = points_for_pinn_model.cpu().detach().numpy()
y_train = thermal_conductivity_equation([points_for_pinn_model[:, 0], points_for_pinn_model[:, 1]]).cpu().detach().numpy()

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

svr = SVR(kernel="rbf", C=1.0, epsilon=0.1, gamma="scale")

svr.fit(X_train, y_train)
# y_pred = svr.predict(X_test_scaled)
# print("MSE:", mean_squared_error(y_test, y_pred))
# print("R^2:", r2_score(y_test, y_pred))

SVR()

In [80]:
# check_current_situation_others(svr, scaler, t=0.0)
# check_current_situation_others(svr, scaler, t=0.01)
# check_current_situation_others(svr, scaler, t=0.02)
# check_current_situation_others(svr, scaler, t=0.03)
# check_current_situation_others(svr, scaler, t=0.04)
# check_current_situation_others(svr, scaler, t=0.05)

# XGBoost

In [81]:
# Example data
xgb_regressor = xgb.XGBRegressor(tree_method='hist', verbosity=2)
xgb_regressor.fit(X_train, y_train)

[17:16:44] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (100, 2, 200).


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [82]:
# check_current_situation_xgb(xgb_regressor, t=0)
# check_current_situation_xgb(xgb_regressor, t=0.01)
# check_current_situation_xgb(xgb_regressor, t=0.02)
# check_current_situation_xgb(xgb_regressor, t=0.03)
# check_current_situation_xgb(xgb_regressor, t=0.04)
# check_current_situation_xgb(xgb_regressor, t=0.05)

# Gaussian Processes Regression

In [83]:
kernel = C(1.0, (1e-3, 1e1)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) \
         + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
gp.fit(X_train, y_train)
# y_pred, sigma = gp.predict(X_test, return_std=True)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10, normalize_y=True)

In [84]:
# check_current_situation_xgb(gp, t=0)
# check_current_situation_xgb(gp, t=0.01)
# check_current_situation_xgb(gp, t=0.02)
# check_current_situation_xgb(gp, t=0.03)
# check_current_situation_xgb(gp, t=0.04)
# check_current_situation_xgb(gp, t=0.05)

# Metrics Computation

In [85]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    median_absolute_error, 
    mean_absolute_percentage_error
)

def evaluate_models(models_dict, X_test, y_test, device='cpu'):
    """
    Evaluates a mix of PyTorch and Scikit-Learn models.
    
    Parameters:
    - models_dict: Dictionary {'ModelName': model_object}
    - X_test: Test features (numpy array, pandas DF, or torch Tensor)
    - y_test: Test targets (numpy array, pandas Series, or torch Tensor)
    - device: 'cpu' or 'cuda' (for PyTorch model)
    
    Returns:
    - pd.DataFrame: Rows are metrics, Columns are model names.
    """
    results = {}
    
    # Ensure y_test is a flat numpy array for scoring
    if torch.is_tensor(y_test):
        y_true = y_test.detach().cpu().numpy().flatten()
    elif isinstance(y_test, (pd.DataFrame, pd.Series)):
        y_true = y_test.to_numpy().flatten()
    else:
        y_true = np.array(y_test).flatten()

    for name, model in models_dict.items():
        y_pred = None
        
        # --- 1. Generate Predictions ---
        
        # Check if the model is a PyTorch Neural Network
        if isinstance(model, nn.Module):
            model.eval() # Set to evaluation mode (disable dropout/batchnorm)
            
            # Prepare input tensor
            if torch.is_tensor(X_test):
                X_tensor = X_test.to(device)
            elif isinstance(X_test, (pd.DataFrame, pd.Series)):
                X_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
            else:
                X_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
            
            with torch.no_grad(): # Disable gradient calculation
                raw_pred = model(X_tensor)
                # Convert back to CPU numpy and flatten to 1D array
                y_pred = raw_pred.cpu().numpy().flatten()
                
        # Assume Scikit-Learn (or compatible API like XGBoost/LGBM)
        else:
            # sk-learn expects numpy/pandas
            if torch.is_tensor(X_test):
                X_input = X_test.cpu().numpy()
            else:
                X_input = X_test
                
            y_pred = model.predict(X_input).flatten()

        # --- 2. Calculate Metrics ---
        
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        # Extra Metric 1: Median Absolute Error (Robust to outliers)
        medae = median_absolute_error(y_true, y_pred)
        
        # Extra Metric 2: MAPE (Mean Absolute Percentage Error)
        # Note: Can be very high if y_true contains zeros
        mape = mean_absolute_percentage_error(y_true, y_pred)

        results[name] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'MedAE': medae,
            'MAPE': mape
        }

    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

In [86]:
pinn = PINN()
checkpoint = torch.load(f"pinn_model_weights_{N}.pth", map_location='cpu')
pinn.load_state_dict(checkpoint)
print(f"pinn_model_weights_{N}.pth")

pinn_model_weights_100.pth


In [93]:
x_physics_test = torch.load(f'data/x_{1000}.pt').to(DEVICE)
time_physics_test = torch.load(f'data/t_{1000}.pt').to(DEVICE)

sol = thermal_conductivity_equation([time_physics_test, x_physics_test]).detach().numpy()
X_test = torch.stack((time_physics_test, x_physics_test),axis=-1).detach().numpy()
y_test = sol

In [94]:
my_models = {
    'PINN': pinn,
    'XGB': xgb_regressor,
    'SVM': svr,
    'GP': gp
}

df_metrics = evaluate_models(my_models, X_test, y_test)

In [95]:
df_metrics

,PINN,XGB,SVM,GP
MSE,0.006954,0.046457,0.060662,0.010925
RMSE,0.083391,0.215538,0.246296,0.104521
MAE,0.053920,0.090199,0.144038,0.056662
R2,0.958875,0.725265,0.641257,0.935394
MedAE,0.030693,0.035656,0.084777,0.027028
MAPE,0.032552,0.052166,0.115634,0.031805
